# Tutoriel clonage de voix

In [1]:
#@title Installation des bibliothèques nécessaires
!source .venv/bin/activate
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
#@title Importation des bibliothèques

import librosa
import soundfile
import numpy as np
from scipy.signal import lfilter
from IPython.display import Audio
import pandas as pd

import os

In [3]:
!ls data/mozilla/cv-corpus-17.0-delta-2024-03-15/en/clips/ | head -n 5

common_voice_en_39586336.mp3
common_voice_en_39586337.mp3
common_voice_en_39586338.mp3
common_voice_en_39586339.mp3
common_voice_en_39586340.mp3
ls: write error: Broken pipe


In [6]:
DATA_PATH = "./data/mozilla/cv-corpus-17.0-delta-2024-03-15/en/"
MP3_PATH = os.path.join(DATA_PATH, "clips/")

In [23]:
df = pd.read_csv(os.path.join(DATA_PATH, "other.tsv"), sep='\t')
df["path"] = MP3_PATH + df["path"]
df

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,7cd86f64838a30eee02cef8f3137bc5d0899d101765754...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,e392b2880b176875ab0676e9fc7298e6c5050e4bed884b...,The message is then read off in rows.,NaN,0,0,fifties,male_masculine,"United States English,Upstate New York",NaN,en,NaN
1,7cd86f64838a30eee02cef8f3137bc5d0899d101765754...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,e39af38b03e30387893a751195054b5d581ddbb8677102...,"In the days when the judges judged, there was ...",NaN,0,0,fifties,male_masculine,"United States English,Upstate New York",NaN,en,NaN
2,7cd86f64838a30eee02cef8f3137bc5d0899d101765754...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,e3a2159c9d212503b24ebab6f7d41cafb0ae9efd8da2a7...,"Edwards was born in Shaw, Mississippi.",NaN,0,0,fifties,male_masculine,"United States English,Upstate New York",NaN,en,NaN
3,7cd86f64838a30eee02cef8f3137bc5d0899d101765754...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,e366873756baac1a5263b4938907ee1fed0e35f9aca7ee...,The qualifications for the award are determine...,NaN,0,0,fifties,male_masculine,"United States English,Upstate New York",NaN,en,NaN
4,7cd86f64838a30eee02cef8f3137bc5d0899d101765754...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,e39a98dd40b15da9aa5ec7f3faf4f98afe26cbe969a39f...,"Hageman, an early settler.",NaN,0,0,fifties,male_masculine,"United States English,Upstate New York",NaN,en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41089,5745495009c5bd49afa2b21b0f0e12cc414f6e7ea7faa3...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,ea1bdce697e60f9f1888e89622b99e14971d67543897eb...,Railroad Museum of Pennsylvania official website,NaN,0,0,NaN,NaN,NaN,NaN,en,NaN
41090,5745495009c5bd49afa2b21b0f0e12cc414f6e7ea7faa3...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,ea1fc6f434affb3ce70b149c9f9e8c541844f727b5fa74...,She then retired from politics and took up res...,NaN,0,0,NaN,NaN,NaN,NaN,en,NaN
41091,5745495009c5bd49afa2b21b0f0e12cc414f6e7ea7faa3...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,ea2cbca3b591f70bcd2a860c0d017a72f86ea6b2496bfb...,It is based in Paris.,NaN,0,0,NaN,NaN,NaN,NaN,en,NaN
41092,5745495009c5bd49afa2b21b0f0e12cc414f6e7ea7faa3...,./data/mozilla/cv-corpus-17.0-delta-2024-03-15...,ea2c69bce8696267f7f39e18bb9c5607e971db573c8eef...,"That year, he moved to the City University of ...",NaN,0,0,NaN,NaN,NaN,NaN,en,NaN


In [21]:
df[df["gender"].str.contains("female") == True]["path"].iloc[0]

'common_voice_en_39587946.mp3'

In [29]:
#@title Chargement des échantillons audio

source_sample = df[df["gender"].str.contains("female") == True].iloc[0]
target_sample = df.iloc[2]

# Charger l'audio de la voix source et de la voix cible
source_audio, source_sr = librosa.load(source_sample["path"], sr=None)
target_audio, target_sr = librosa.load(target_sample["path"], sr=None)

# Assurez-vous que les deux échantillons audio ont le même taux d'échantillonnage
if source_sr != target_sr:
    target_audio = librosa.resample(target_audio, target_sr, source_sr)
    print("Resampling : done")


In [33]:
print(source_sample["sentence"])
Audio(data=source_audio, rate=source_sr)

Norris Green Park is situated between Broad Lane and Lorenzo Drive.


In [32]:
print(target_sample["sentence"])
Audio(data=target_audio, rate=target_sr)

Edwards was born in Shaw, Mississippi.


In [36]:
#@title Extraction des caractéristiques vocales

# Extraire les caractéristiques pertinentes (par exemple, MFCCs) de la voix source et de la voix cible
source_features = librosa.feature.mfcc(y=source_audio, sr=source_sr)
target_features = librosa.feature.mfcc(y=target_audio, sr=source_sr)

source_features.shape, target_features.shape

((20, 374), (20, 273))

In [41]:
#@title Modélisation de la conversion vocale (simplifié)

# Cette étape est conceptuelle et nécessiterait un modèle de conversion vocale entraîné
# Ici, nous utilisons un filtrage simple pour illustrer l'idée

# Appliquer un filtre basé sur les caractéristiques de la voix cible à l'audio source
converted_audio = lfilter(target_features.mean(axis=1), [1.0], source_audio)

In [42]:
# Écouter l'audio converti (si l'environnement le permet)
Audio(converted_audio, rate=source_sr)

In [ ]:
#@title Écoute et sauvegarde de l'audio converti


# Sauvegarder l'audio converti
soundfile.write("voix_convertie.wav", converted_audio, source_sr)